<a href="https://colab.research.google.com/github/plus2net/numpy/blob/main/numpy_9_random_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://www.plus2net.com/images/top2.jpg)        Read more on [Random Generator ](https://www.plus2net.com/python/numpy-random-generator.php) | [ Numpy ](https://www.plus2net.com/python/numpy.php)

In [1]:
import numpy as np

# Modern API (recommended)
rng = np.random.default_rng(seed=42)  # PCG64 by default
print(rng.integers(0, 10, size=5))
print(rng.normal(0, 1, size=3))

# Legacy (still works, not recommended for new code)
# np.random.seed(42)
# np.random.randint(0, 10, 5)

[0 7 6 4 4]
[ 0.94056472 -1.95103519 -1.30217951]


In [2]:
from numpy.random import Generator, PCG64, Philox

rng_pcg = Generator(PCG64(12345))
rng_philox = Generator(Philox(2025))

print(rng_pcg.integers(10, size=4))
print(rng_philox.uniform(size=3))

[6 2 7 3]
[0.06968444 0.75389778 0.45656435]


In [3]:
from numpy.random import SeedSequence, PCG64, Generator

ss = SeedSequence(2025)
# Spawn 3 independent child sequences
child_ss = ss.spawn(3)
streams = [Generator(PCG64(s)) for s in child_ss]

# Each stream is independent and reproducible
for i, rng in enumerate(streams, 1):
    print('stream', i, rng.integers(0, 100, size=3))

stream 1 [87 99 72]
stream 2 [53 71 62]
stream 3 [ 5 72 24]


In [4]:
# Example sketch (no actual multiprocessing shown)
base = SeedSequence(123)
children = base.spawn(4)
rngs = [Generator(PCG64(s)) for s in children]

# worker k uses rngs[k] exclusively

In [5]:
import numpy as np

rng = np.random.default_rng(7)
state = rng.bit_generator.state            # dict

# Generate some numbers
a = rng.standard_normal(5)

# Restore exact state later
rng2 = np.random.default_rng()
rng2.bit_generator.state = state
b = rng2.standard_normal(5)

print(np.allclose(a, b))  # True (same continuation)

True


In [6]:
rng = np.random.default_rng(101)
print(rng.choice(['A','B','C'], size=5, replace=True, p=[0.2,0.5,0.3]))
print(rng.normal(loc=10, scale=2, size=(2,3)))
print(rng.poisson(lam=3.5, size=4))

['C' 'B' 'C' 'B' 'B']
[[10.73464275 13.42078858 12.12159568]
 [11.41527804 11.37549878  8.27286509]]
[2 5 2 7]


In [8]:
import json, numpy as np

rng = np.random.default_rng(2026)
state = rng.bit_generator.state
with open('rng_state.json', 'w') as f:
    json.dump(state, f)

# Later:
with open('rng_state.json') as f:
    state2 = json.load(f)
rng2 = np.random.default_rng()
rng2.bit_generator.state = state2

In [9]:
# 1) Create three independent RNG streams using SeedSequence.spawn and draw 2 Poisson numbers from each.
from numpy.random import SeedSequence, PCG64, Generator
ss = SeedSequence(1234)
streams = [Generator(PCG64(s)) for s in ss.spawn(3)]
for i, r in enumerate(streams, 1):
    print(i, r.poisson(2.5, size=2))

# 2) Save RNG state after 5 draws, restore it, and confirm the next 5 draws match
rng = np.random.default_rng(77)
first = rng.normal(size=5)
st = rng.bit_generator.state
cont1 = rng.normal(size=5)

rng2 = np.random.default_rng()
rng2.bit_generator.state = st
cont2 = rng2.normal(size=5)
print(np.allclose(cont1, cont2))

# 3) Replace legacy np.random calls with Generator equivalents in a toy script.

# 4) Simulate a biased 6-sided die with probabilities and verify empirical frequencies.
rng = np.random.default_rng(9)
p = np.array([0.10, 0.15, 0.20, 0.25, 0.20, 0.10])
rolls = rng.choice(np.arange(1,7), size=10_000, p=p)
print(np.round(np.bincount(rolls, minlength=7)[1:] / rolls.size, 3))

1 [4 1]
2 [3 4]
3 [0 2]
True
[0.098 0.153 0.196 0.257 0.2   0.096]
